In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np

dataToLoad = "newghUTF8.csv"

grubhubDirty = pd.read_csv(dataToLoad, encoding='utf8')

FileNotFoundError: [Errno 2] No such file or directory: 'newghUTF8.csv'

In [ ]:
grubhubDirty.head(3)

In [ ]:
# Getting later grouped by out of the way
groupedByEstablishment = grubhubDirty.groupby(['streetname', 'establishment'])
groupedMeanDF = groupedByEstablishment.mean()
groupedMeanDF.head()

# Quick Transformations

In [ ]:
ordersPerEstablishment = groupedByEstablishment["tip"].count()
ordersPerEstablishment.rename("ordersFromEstablishment")
ordersPerEstablishmentDF = pd.merge(grubhubDirty, ordersPerEstablishment, how = "inner", on = "streetname")
ordersPerEstablishmentDF = ordersPerEstablishmentDF.rename(columns = {'tip_x': 'tip', 'tip_y': 'aggOrders'})
grubhubClean = ordersPerEstablishmentDF
#grubhubClean.head(3)

In [ ]:
groupedByEstablishment = grubhubClean.groupby(['streetname', 'establishment'])
groupedMeanDF = groupedByEstablishment.mean()
groupedMeanDF.head()

# Summary Tables

In [ ]:
totalEstablishments = len(grubhubClean["streetname"].unique())
totalOrders = len(grubhubDirty)
totalMiles = grubhubClean["miles"].sum()
totalMilePay = grubhubClean["mileagePay"].sum()
totalTimePay = grubhubClean["timePay"].sum()
totalBonusPay = grubhubClean["bonus"].sum()
totalTips = grubhubClean["tip"].sum()
totalPay = grubhubClean["total"].sum()
grubhubShare = totalPay - totalTips

In [ ]:
basicSummaryDF = pd.DataFrame({
    
    "Total Establishments": [totalEstablishments],
    "Total Orders": [totalOrders],
    "Total Miles": [totalMiles],
    "Total Pay": [totalPay],
    "Total Tips": [totalTips],
    "GrubHub Share": [grubhubShare],
    "Total Mile Pay": [totalMilePay],
    "Total Time Pay": [totalTimePay],
    "Total Bonus": [totalBonusPay]
    
})

basicSummaryDF

In [ ]:
timePayPerOrder = totalTimePay/totalOrders
milesPerOrder = totalMiles/totalOrders
payPerMile = totalMilePay/totalMiles
bonusPerOrder = totalBonusPay/totalEstablishments
averageTip = grubhubClean["tip"].mean()
averageOrderTotal = totalPay/totalOrders
grubhubAverage = averageOrderTotal - averageTip
milePayPerOrder = totalMilePay/totalOrders
timePayPerOrder = grubhubClean["timePay"].mean()

In [ ]:
relativeSummaryDF = pd.DataFrame({
    
    "Average Total Pay per Order": [averageOrderTotal],
    "Average Tip": [averageTip],
    "GrubHub Average": [grubhubAverage],
    "Miles Per Order": [milesPerOrder],
    "Pay Per Mile": [payPerMile],
    "Mile Pay Per Order": [milePayPerOrder],
    "Time Pay Per Order": [timePayPerOrder],
    "Bonus Per Order": [bonusPerOrder]    
    
})
relativeSummaryDF

# 1. Frequency Analysis

In [ ]:
frequentOrders = groupedMeanDF.loc[groupedMeanDF["aggOrders"] >= 10]
frequentOrders

In [ ]:
occasionalOrders = groupedMeanDF[(groupedMeanDF["aggOrders"] >= 5) & (groupedMeanDF["aggOrders"] < 10)]
occasionalOrders.head()

In [ ]:
infrequentOrders = groupedMeanDF[(groupedMeanDF["aggOrders"] < 5) & (groupedMeanDF["aggOrders"] > 1)]
infrequentOrders.head()

# 2. Bonus Payments

In [ ]:
needBonusDF = grubhubClean[grubhubClean["bonus"] > 0]
#needBonusDF.head()

In [ ]:
numberOfBonusOrders = len(needBonusDF)
averageBonus = totalBonusPay/numberOfBonusOrders
highestBonus = needBonusDF["bonus"].max()
percentWithBonus = numberOfBonusOrders/totalOrders
lowestBonus = needBonusDF["bonus"].min()

In [ ]:
bonusBasicSummary = pd.DataFrame({
    
    "Number Of Bonus Orders": [numberOfBonusOrders],
    "% Orders With Bonus": [percentWithBonus],
    "Highest Bonus": [highestBonus],
    "Lowest Bonus": [lowestBonus],
    "Average Bonus Per Order; Non-Null": [averageBonus],
    "Bonus Per Order; All":[bonusPerOrder]
    
})
bonusBasicSummary

# 2. More Digging into Bonuses

In [ ]:
biggestBonusesDF = needBonusDF[needBonusDF["bonus"] >= 5]
biggestBonusesDF

In [ ]:
littleBonusesDF = needBonusDF[needBonusDF["bonus"] < 1]
littleBonusesDF.head()

In [ ]:
bigBonusResturants = groupedByEstablishment.filter(lambda x: x['bonus'].mean() >= 2.00)
bigBonusResturants.head()

In [ ]:
bigBonusResturantsMean = bigBonusResturants.groupby(['streetname', 'establishment']).mean()
bigBonusResturantsMean

# 3. Tipping Analysis

In [ ]:
bigTipDF = grubhubClean.loc[grubhubClean["tip"] >10]
bigTipDF.head()

In [ ]:
bigTipResturants = groupedByEstablishment.filter(lambda x: x['tip'].mean() >= 10.00)
mediumTipResturants = groupedByEstablishment.filter(lambda x: ((x['tip'].mean() >= 5.00) & (x['tip'].mean() < 10.00)))
#mediumTipResturants.head()
smallTipResturants = groupedByEstablishment.filter(lambda x: x['tip'].mean() < 5.00)
#smallTipResturants.head()
bigTipResturants

In [ ]:
bigTipLocCount = len(bigTipResturants["establishment"].unique())
mediumTipLocCount = len(mediumTipResturants["establishment"].unique())
smallTipLocCount = len(smallTipResturants["establishment"].unique())

In [ ]:
tipBasicSummary = pd.DataFrame({
    
    "Average Tips; All": [averageTip],
    "Establishments, Mean Tip > 10": [bigTipLocCount],
    "Establishments, Mean Tip 5 to 10": [mediumTipLocCount],
    "Establishments, Mean Tip < 5": [smallTipLocCount]
    
})
tipBasicSummary

# 3. More Digging into Tips

In [ ]:
mediumTipDF = mediumTipResturants.groupby(['streetname', 'establishment'])
mediumTipMeanDF = mediumTipDF.mean()
mediumTipMeanDF.sort_values(["tip"],ascending=False).head(10)

In [ ]:
smallTipDF = smallTipResturants.groupby(['streetname', 'establishment'])
smallTipMeanDF = smallTipDF.mean()
smallTipMeanDF.sort_values(["tip"],ascending=False).tail()

#Editors Note, Bella order was canceled.

# 4. Mileage Analysis

In [ ]:
longDistanceResturaunts = groupedByEstablishment.filter(lambda x: x['miles'].mean() >= 10.00)
longDistanceResturaunts.head(3) 

In [ ]:
mediumDistanceResturuants = groupedByEstablishment.filter(lambda x: ((x['miles'].mean() >= 5.00) & (x['miles'].mean() < 10.00)))
mediumDistanceResturuants.head(3)

In [ ]:
mediumDistanceDF = mediumDistanceResturuants.groupby(['streetname', 'establishment'])
mediumDistanceMeanDF = mediumDistanceDF.mean()
mediumDistanceMeanDF.sort_values(["miles"],ascending=False).head(10)

In [ ]:
shortDistanceResturaunts = groupedByEstablishment.filter(lambda x: x['miles'].mean() < 5.00)
shortDistanceResturaunts.head(3)

In [ ]:
shortDistanceDF = shortDistanceResturaunts.groupby(['streetname', 'establishment'])
shortDistanceMeanDF = shortDistanceDF.mean()
shortDistanceMeanDF.sort_values(["miles"],ascending=False).tail(15)

In [ ]:
shortDistanceMeanDF.describe()

# Graphs

In [ ]:
x_axis = grubhubClean["total"]
y_axis = grubhubClean["miles"]

correlation_coeff = st.pearsonr(x_axis, y_axis)
print(f"The correlation coefficient: {round(correlation_coeff[0],2)}.")

#Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regression_value = slope * x_axis + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.scatter(x_axis, y_axis)
plt.plot(x_axis, regression_value, "r-")
#plt.annotate(line_equation, (color="red")
corr_coeff_str = "correlation coefficient:" + str(round(correlation_coeff[0],2))
#plt.annotate(corr_coeff_str, (color="red")
#plt.ylim()
plt.title("Total vs Miles")
plt.xlabel("Total")
plt.ylabel("Miles")
plt.show()

In [ ]:
x_axis = grubhubClean['miles']
y_axis = grubhubClean['mileagepay']

correlation_coeff = st.pearsonr(x_axis, y_axis)
print(f"The correlation coefficient: {round(correlation_coeff[0],2)}.")

#Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regression_value = slope * x_axis + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.scatter(x_axis, y_axis)
plt.plot(x_axis, regression_value, "r-")
#plt.annotate(line_equation, (color="red")
corr_coeff_str = "correlation coefficient:" + str(round(correlation_coeff[0],2))
#plt.annotate(corr_coeff_str, (color="red")
#plt.ylim()
plt.title("Miles vs Mileage Pay")
plt.xlabel("Miles")
plt.ylabel("Mileage Pay")
plt.show()

In [ ]:
x_axis = grubhubClean['tip']
y_axis = grubhubClean['total']

correlation_coeff = st.pearsonr(x_axis, y_axis)
print(f"The correlation coefficient: {round(correlation_coeff[0],2)}.")

#Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regression_value = slope * x_axis + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.scatter(x_axis, y_axis)
plt.plot(x_axis, regression_value, "r-")
#plt.annotate(line_equation, (color="red")
corr_coeff_str = "correlation coefficient:" + str(round(correlation_coeff[0],2))
#plt.annotate(corr_coeff_str, (color="red")
#plt.ylim()
plt.title("Tip vs Total")
plt.xlabel("Tip")
plt.ylabel("Total")
plt.show()

In [ ]:
x_axis = grubhubClean['timepay']
y_axis = grubhubClean['total']

correlation_coeff = st.pearsonr(x_axis, y_axis)
print(f"The correlation coefficient: {round(correlation_coeff[0],2)}.")

#Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regression_value = slope * x_axis + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.scatter(x_axis, y_axis)
plt.plot(x_axis, regression_value, "r-")
#plt.annotate(line_equation, (color="red")
corr_coeff_str = "correlation coefficient:" + str(round(correlation_coeff[0],2))
#plt.annotate(corr_coeff_str, (color="red")
#plt.ylim()
plt.title("Time vs Total")
plt.xlabel("Time")
plt.ylabel("Total")
plt.show()

In [ ]:
x_axis = grubhubClean['bonus']
y_axis = grubhubClean['total']

correlation_coeff = st.pearsonr(x_axis, y_axis)
print(f"The correlation coefficient: {round(correlation_coeff[0],2)}.")

#Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regression_value = slope * x_axis + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.scatter(x_axis, y_axis)
plt.plot(x_axis, regression_value, "r-")
#plt.annotate(line_equation, (color="red")
corr_coeff_str = "correlation coefficient:" + str(round(correlation_coeff[0],2))
#plt.annotate(corr_coeff_str, (color="red")
#plt.ylim()
plt.title("Bonus vs Total")
plt.xlabel("Bonus")
plt.ylabel("Total")
plt.show()

In [ ]:
# Once More, with grouped means instead

In [ ]:
x_axis = groupedMeanDF['bonus']
y_axis = groupedMeanDF['total']

correlation_coeff = st.pearsonr(x_axis, y_axis)
print(f"The correlation coefficient: {round(correlation_coeff[0],2)}.")

#Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regression_value = slope * x_axis + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.scatter(x_axis, y_axis)
plt.plot(x_axis, regression_value, "r-")
#plt.annotate(line_equation, (color="red")
corr_coeff_str = "correlation coefficient:" + str(round(correlation_coeff[0],2))
#plt.annotate(corr_coeff_str, (color="red")
#plt.ylim()
plt.title("Means: Bonus vs Total")
plt.xlabel("Bonus")
plt.ylabel("Total")
plt.show()

In [ ]:
x_axis = grubhubClean['total']
y_axis = grubhubClean['rating']

correlation_coeff = st.pearsonr(x_axis, y_axis)
print(f"The correlation coefficient: {round(correlation_coeff[0],2)}.")

#Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regression_value = slope * x_axis + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.scatter(x_axis, y_axis)
plt.plot(x_axis, regression_value, "r-")
#plt.annotate(line_equation, (color="red")
corr_coeff_str = "correlation coefficient:" + str(round(correlation_coeff[0],2))
#plt.annotate(corr_coeff_str, (color="red")
#plt.ylim()
plt.title("Rating vs Total")
plt.xlabel("Total")
plt.ylabel("Rating")
plt.show()

In [ ]:
x_axis = grubhubClean['tip']
y_axis = grubhubClean['rating']

correlation_coeff = st.pearsonr(x_axis, y_axis)
print(f"The correlation coefficient: {round(correlation_coeff[0],2)}.")

#Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regression_value = slope * x_axis + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.scatter(x_axis, y_axis)
plt.plot(x_axis, regression_value, "r-")
#plt.annotate(line_equation, (color="red")
corr_coeff_str = "correlation coefficient:" + str(round(correlation_coeff[0],2))
#plt.annotate(corr_coeff_str, (color="red")
#plt.ylim()
plt.title("Tip vs Rating")
plt.xlabel("Tip")
plt.ylabel("Rating")
plt.show()

In [ ]:
x_axis = grubhubClean['miles']
y_axis = grubhubClean['rating']

correlation_coeff = st.pearsonr(x_axis, y_axis)
print(f"The correlation coefficient: {round(correlation_coeff[0],2)}.")

#Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regression_value = slope * x_axis + intercept
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


plt.scatter(x_axis, y_axis)
plt.plot(x_axis, regression_value, "r-")
#plt.annotate(line_equation, (color="red")
corr_coeff_str = "correlation coefficient:" + str(round(correlation_coeff[0],2))
#plt.annotate(corr_coeff_str, (color="red")
#plt.ylim()
plt.title("Miles vs Rating")
plt.xlabel("Miles")
plt.ylabel("Rating")
plt.show()